## Assignment 1. Neural Text Classification
## CS310 Natural Language Processing

**Total points**: 50

You should roughtly follow the structure of the notebook. Add additional cells if you feel needed. 

You can (and you should) re-use the code from Lab 2. 

Make sure your code is readable and well-structured.

### 0. Import Necessary Libraries

In [8]:
import json
import re
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchtext.vocab import build_vocab_from_iterator
import time
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
import jieba
import paddle

### 1. Data Processing

In [9]:
class myDataset(Dataset):
    def __init__(self, file_path):
        self.data = self._process_data(file_path)

    def _process_data(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        processed_data = []
        for line in lines:
            json_data = json.loads(line)
            sentence = json_data['sentence']
            choices = json_data['choices']
            label = json_data['label'][0]
            id = json_data['id']

            processed_sentence = self._tokenize(sentence)
            processed_choices = [self._tokenize(choice) for choice in choices]

            processed_data.append({
                'sentence': processed_sentence,
                'label': label
            })
        return processed_data

    def _tokenize(self, text):
        # # Tokenize text by treating each single Chinese character as a token
        # tokens = re.findall(r'[\u4e00-\u9fff]', text)
        # return ''.join(tokens)
        #Improved
        chinese_pattern = re.compile(r'[\u4e00-\u9fff]')
        digit_pattern = re.compile(r'\d+')
        english_word_pattern = re.compile(r'[a-zA-Z]+')
        # 匹配除了中英文数字空格之外的特殊字符
        punctuation_pattern = re.compile(r'[^\u4e00-\u9fff\da-zA-Z\s]')
        tokens= re.findall(r'[\u4e00-\u9fff]|\d+|[a-zA-Z]+|[^\u4e00-\u9fff\da-zA-Z\s]', text)
        return ''.join(tokens)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

# Example usage
train_dataset = myDataset('train.jsonl')
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
train_iterator = iter(train_dataloader)


count = 0
for item in train_dataloader:
    print(item)
    count += 1
    if count > 7:
        break

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



AttributeError: 'dict' object has no attribute 'p'

### 2. Build the Model

In [ ]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(Model, self).__init__()

        self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim, sparse=True)
        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, text):
        embedded = self.embedding(text)
        output = self.fc(embedded)
        return output

# Example usage
vocab_size = 10000
embedding_dim = 100
hidden_dim = 256
num_classes = 2

model = Model(vocab_size, embedding_dim, hidden_dim, num_classes)

### 3. Train and Evaluate

### 4. Explore Word Segmentation